本笔记参考了下面的书籍、文献、博客或者官方说明：
* TensorFlow2官方文档：https://tensorflow.google.cn/
* 简单粗暴TensorFlow 2：https://github.com/snowkylin/tensorflow-handbook
* TensorFlow 2.0 学习笔记：https://zhuanlan.zhihu.com/p/74441082

未注明出处的代码示例，`大概`就是我自己编的，`大概`的意思就是也有极小的概率是忘记注明了。。。

In [1]:
import tensorflow as tf

# Stateful Container

### Trackable

In [2]:
from tensorflow.python.training.tracking.base import Trackable

In [3]:
x = Trackable()
y = Trackable()
x._track_trackable(y, 'ccc') # x引用y，并且叫该引用命名为'ccc'，或者说x依赖y

In [4]:
x._lookup_dependency('ccc') is y  # 返回名称为'ccc'的引用

True

In [5]:
y

In [6]:
x._lookup_dependency('ccc')

In [7]:
del y

In [8]:
x._lookup_dependency('ccc')

可以看到删除y之后，不影响x对其引用。因此只要根节点x没有被回收，那么x所依赖的对象就不会被回收。

### AutoTrackable
AutoTrackabke类继承Trackable类，通过`__setattr__`和`__getattr__`属性拦截访问和设置新属性（访问和建立依赖关系）。

In [9]:
from tensorflow.python.training.tracking.tracking import AutoTrackable

In [10]:
x = AutoTrackable()
y = AutoTrackable()
x.ccc = y

In [11]:
x._lookup_dependency('ccc') is y

True

In [12]:
v = tf.Variable([1,2,3])

In [13]:
x.vvv = v

In [14]:
x._unconditional_checkpoint_dependencies

[TrackableReference(name='ccc', ref=<tensorflow.python.training.tracking.tracking.AutoTrackable object at 0x13d72f2b0>),
 TrackableReference(name='vvv', ref=<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>)]

In [15]:
v

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

### 可以被保存的对象
**tf.Variable和MutableHashTable**  
tf.Variable类和MutableHashTable类是可以被保存的对象(用于tf.train.Checkpoint)，这两个类继承自Trackable类，并且覆盖了`_gather_saveables_for_checkpoint`方法，用tf.train.Checkpoint来保存。

In [16]:
from tensorflow.python.ops.lookup_ops import MutableHashTable

In [17]:
x._gather_saveables_for_checkpoint()

{}

In [18]:
x.vvv._gather_saveables_for_checkpoint()

{'VARIABLE_VALUE': <tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>}

实际上，Checkpoint使用了ObjectGraphView类，遍历整个DAG节点，并调用`_gather_saveables_for_checkpoint`方法类收集可以被保存的对象以及它们的依赖关系并存储。

### Restore-on-Creation

In [19]:
class MyModule(tf.Module):
    def assign(self, init=tf.constant([1., 2., 3.]), name=None):
        with self.name_scope:
          self.w = tf.Variable(init)
    def operate(self, value):
        self.w.assign_add(value)

m = MyModule(name='test')
m.assign()
m.operate([1., 1., 1.])
m.w

<tf.Variable 'test/Variable:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

In [20]:
ckpt = tf.train.Checkpoint(module=m)
ckpt.save('data/ckpt.save.test')

'data/ckpt.save.test-1'

In [21]:
module = MyModule(name='test')
try:
    module.w
except AttributeError as e:
    print("w doesn't exist.")
else:
    print("w already exists.")

w doesn't exist.


由于没用调用assign方法，可以看到w属性是不存在的。

In [22]:
ckpt = tf.train.Checkpoint(module=module)
ckpt.restore(tf.train.latest_checkpoint('data'))

In [23]:
try:
    module.w
except AttributeError as e:
    print("w doesn't exist.")
else:
    print("w already exists.")

w doesn't exist.


可以看到由于w属性没有建立，因此restore之后，w依然是不存在的。但是当调用assign方法建立w属性的时候，restore就会起作用了，可以看到结果是restore得到的结果，并不是assign的参数所指定的`tf.constant([1., 1., 1.])`。  

**Restore-on-Creation机制就是在权重没有建立时，暂时不加载checkpoint保存的权重，一旦建立，则立即加载。**

In [24]:
module.assign(tf.constant([1., 1., 1.]))
module.w  # so you see...

<tf.Variable 'test/Variable:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

### tf.Module
`tf.variables`：收集所有变量；  
`tf.trainable_variables`：收集所有可训练的变量；  
`tf.submodules`：收集所有子模块。

> You can enter the name scope explicitly using `with self.name_scope:` or you can annotate methods(apart from `__init__`) with `@tf.Module.with_name_scope`.

注意使用`@tf.Module.with_name_scope`或者`with self.name_scope`，必须在`__init__`中调用`super().__init__`，以此来调用`tf.Module`类的构建函数`__init__`

In [25]:
class Dense(tf.Module):
  def __init__(self, input_features, output_features, name=None):
    super(Dense, self).__init__(name=name)
    with self.name_scope:
      self.w = tf.Variable(tf.random.normal([input_features, output_features], name='w'))
      self.b = tf.Variable(tf.zeros([output_features,]), name='b')
  @tf.Module.with_name_scope
  def __call__(self, x):
    self.test = tf.Variable([2.,3.], name='ahaha')
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

d = Dense(input_features=5, output_features=3)
d(tf.ones([6, 5]))

<tf.Tensor: id=98, shape=(6, 3), dtype=float32, numpy=
array([[1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ]], dtype=float32)>

In [26]:
d.variables[0].name

'dense/b:0'

In [27]:
d.name_scope.name

'dense/'

In [28]:
d.name

'dense'

In [29]:
d.test

<tf.Variable 'dense/ahaha:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>

# tf.function

### 基本特征

* tf.function 装饰器返回的是def_function.Function对象；
* Function对象是由一个个的ConcreteFunction函数组成；ConcreteFunction对象是由包含了FunctionGraph和structured_input_signature；
* FunctionGraph是tf.Graph的子类，strucured_input_signature是函数签名；
* 如果传入的参数是一个python值，则会对每一个遇到的pyhon值创建一个ConcreteFunction，实际上python值会成为Graph的一个固定的值，如果创建ConcreteFunction时，参数是一个python的引用，则此时引用的值就被固定在Graph中；
* 这也说明，如果是参数是可变了python值，那么，在函数中就不能运行原处改变的操作，因为该值已经被固定在Graph中了；

### 运行过程

1. 运行函数的每一行代码，代码分为两类：
  * 纯python代码；
  * tensorflow代码，如`tf.add`，以及可以转换为计算节点的python代码；  
运行的结果就是：纯python代码会与运行普通的python代码相同，tensorflow代码与可以转换为计算节点的python代码会构建为计算图。
2. 运行计算图一次
3. 基于函数的名字和输入的函数参数类型生成一个哈希值，并将计算的计算图缓存到一个哈希表中

**AutoGraph与if，while循环：**  
* for：如果iterable是张量，则转换；
* while：如果while条件是张量，则转换。

### 实例

In [30]:
@tf.function
def add(x, y):
    return tf.add(x, y)

In [31]:
add(tf.random.normal((2, 3)), tf.random.normal((3,)))

<tf.Tensor: id=119, shape=(2, 3), dtype=float32, numpy=
array([[ 3.5347419 ,  1.4418387 ,  0.48181692],
       [ 4.3130684 , -0.42558745,  0.92748874]], dtype=float32)>

In [32]:
add(tf.random.normal((2, 6)), tf.random.normal((6,)))

<tf.Tensor: id=138, shape=(2, 6), dtype=float32, numpy=
array([[ 1.4450588 , -1.1302978 , -0.365623  ,  0.01166382,  3.4731584 ,
         0.07145485],
       [-0.4821213 ,  0.20358813, -2.4397168 , -1.2038107 ,  1.6287951 ,
         0.8607738 ]], dtype=float32)>

In [33]:
add._list_all_concrete_functions_for_serialization()

In [34]:
add(3,5);add(6,9)

<tf.Tensor: id=164, shape=(), dtype=int32, numpy=15>

In [35]:
add._list_all_concrete_functions_for_serialization()[2].structured_input_signature

((6, 9), {})

In [36]:
add._list_all_concrete_functions_for_serialization()[3].structured_input_signature

((3, 5), {})

In [37]:
add._list_all_concrete_functions_for_serialization()[3]()  # 参数是python值所对应的ConcreteFunction函数不需要传入参数了，因为参数值已经固定在里面了

<tf.Tensor: id=165, shape=(), dtype=int32, numpy=8>

In [38]:
add._list_all_concrete_functions_for_serialization()

In [39]:
sig = add._list_all_concrete_functions_for_serialization()[0].structured_input_signature
sig

((TensorSpec(shape=(2, 6), dtype=tf.float32, name='x'),
  TensorSpec(shape=(6,), dtype=tf.float32, name='y')),
 {})

`.get_concrete_function`获取ConcreteFunction，奇怪的是两种方式获得ConcreteFunction并不相等

In [40]:
a = add.get_concrete_function(tf.TensorSpec(shape=[2,6], dtype=tf.float32), tf.TensorSpec(shape=[6,], dtype=tf.float32))

In [41]:
a

In [42]:
add._list_all_concrete_functions_for_serialization()[0]

tf.function只允许在第一次调用函数时，创建tf.Variable；因此典型用法应当是在`__init__`方法中设置权重为`None`，然后在`build`方法中加以判断，如果权重为`None`，则初始化权重。

In [51]:
v = None

def f(x):
    global v
    if v is None:
      v = tf.Variable(x)
    return v
f = tf.function(f)

In [52]:
f._list_all_concrete_functions_for_serialization()

[]

In [53]:
f(tf.constant([2., 3., 4.]))

<tf.Tensor: id=314, shape=(3,), dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

In [54]:
f(tf.constant([2., 3.]))

<tf.Tensor: id=322, shape=(3,), dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

当我把v重新设置成None时，导致再次调用函数f时会试图创建variable，因此抛出异常。

In [55]:
try:
    v = None
    f(tf.constant([1.,2, 3.]))
except ValueError:
    print("ValueError when create variable non-first call")
else:
    print("isn't ok?")

ValueError when create variable non-first call


正确的用法应当是：

In [56]:
class MyModule(tf.Module):
    def __init__(self, name, units=10):
        super(MyModule, self).__init__(name=name)
        self.w = None
        self.b = None
        self.units = units
    @tf.Module.with_name_scope
    def build(self, input_shape):
        if self.w is None:
            self.w = tf.Variable(tf.random.normal([input_shape[-1], self.units]))
        if self.b is None:
            self.b = tf.Variable(tf.random.normal([self.units, ]))
    def call(self, input):
        return tf.matmul(input, self.w) + self.b
    @tf.function
    def __call__(self, input):
        self.build(input.shape)
        return self.call(input)

In [57]:
m = MyModule('testModule')
input = tf.random.normal([5,3])
m(input).shape

TensorShape([5, 10])

In [58]:
m.__call__._list_all_concrete_functions_for_serialization()[0].structured_input_signature

((TensorSpec(shape=(5, 3), dtype=tf.float32, name='input'),), {})

如果注释掉`build`方法中的两个`if`判断语句，导致`ValueError when create variable non-first call`

### 可变类型作为函数的参数

In [59]:
@tf.function
def f(x):
    print(x)
    # 这一行会导致错误，也就是说参数是可变类型的原处操作会导致运行错误
    # x.append(100) 
    return x[-1] + 100

In [60]:
x = [1.,2.]

In [61]:
f(x)

[1.0, 2.0]


<tf.Tensor: id=441, shape=(), dtype=float32, numpy=102.0>

In [62]:
f.get_concrete_function(x)()

<tf.Tensor: id=442, shape=(), dtype=float32, numpy=102.0>

In [63]:
f._list_all_concrete_functions_for_serialization()[0].structured_input_signature

(([1.0, 2.0],), {})

可以看到上面的例子说明：python的可变类型作为参数时，除了不能用原处操作的方法外，其他的和python值作为参数时是相同的。

下面这个例子来自于TensorFlow 2官方文档：

In [64]:
l = [] 
@tf.function 
def f(x): 
  for i in x: 
    l.append(i + 1)    # Caution! Will only happen once when tracing 
f(tf.constant([1, 2, 3])) 
l

[<tf.Tensor 'add:0' shape=() dtype=int32>]

In [65]:
f._list_all_concrete_functions_for_serialization()[0]

In [66]:
l = []
@tf.function
def f(a):
    for i in range(a):
        l.append(0)  # 只会在构建计算图时运行一次
        tf.print(a)  # 会成为计算图的一个计算节点，每次调用都会运行

In [67]:
f(3)
l

3
3
3


[0, 0, 0]

In [68]:
f(3)  # 第二次调用并不会改变list的值，因为第二次只会运行计算图
l

3
3
3


[0, 0, 0]

### 自定义类的序列化

In [70]:
class Person:
    def __init__(self, age):
        self.age = age

@tf.function
def f(year, p):
    print(year)
    return p.age + year

p = Person(100)

In [71]:
f(1, p)

1


<tf.Tensor: id=546, shape=(), dtype=int32, numpy=101>

In [72]:
f(2, p)

2


<tf.Tensor: id=551, shape=(), dtype=int32, numpy=102>

In [73]:
f(2,p)

<tf.Tensor: id=552, shape=(), dtype=int32, numpy=102>

In [81]:
f.get_concrete_function(2,p).structured_input_signature

((2, <tensorflow.python.framework.func_graph.UnknownArgument at 0x13e59f0b8>),
 {})

可能是由于Person类并没有序列化，因此导致`_list_all_concrete_functions_for_serialization`并不能获取`ConcreteFunction`

In [82]:
f._list_all_concrete_functions_for_serialization()

[]

In [91]:
@tf.function
def concat_with_padding():
    x = tf.zeros([5, 10])
    tf.print(x.shape)
    x = x[:4]
    tf.print(x.shape)
    for i in tf.range(4):
        x = tf.concat([x[:i], tf.ones([1, 10])], axis=0) # 循环时张量形状不能改变
        tf.print(x.shape)
        x.set_shape([4, 10])
        tf.print(x.shape)
    return x
concat_with_padding()

TensorShape([5, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])


<tf.Tensor: id=1095, shape=(4, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>